In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'
do_jsd = False
do_mmd = True
save_res = True
top_out_dir = top_evaluation_dir

experiment_name = 'l_gan_emd'
sample_mmd = True

n_samples = 10
ref_pop = 40
sample_pop = 200

mmd_loss = 'emd'

special_tag = 'emd_sample_40_200'
skip = ['test', 'val']

In [5]:
top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [6]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [7]:
if experiment_name.startswith('l_') and 'gan' in experiment_name:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
elif experiment_name.startswith('r_gan'):
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

In [8]:
sample_files = []
if 'gan' in experiment_name:
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
elif 'gmm' in experiment_name:
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

In [9]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [10]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
        f_mmd.flush()
    print

epoch_1
train 317.06 9.86493

epoch_5
train 238.331 10.4511

epoch_10
train 265.5 8.90972

epoch_100
train 282.506 9.18514

epoch_200
train 281.339 7.25771

epoch_300
train 274.543 7.9639

epoch_400
train 271.683 9.46674

epoch_500
train 274.415 6.65581

epoch_600
train 266.91 7.08649

epoch_700
train 272.741 7.52562

epoch_800
train 266.409 12.285

epoch_900
train 266.254 7.61048

epoch_1000
train 272.091 14.4733

epoch_1100
train 261.877 5.11743

epoch_1200
train 262.592 4.54242

epoch_1300
train 257.896 9.03718

epoch_1400
train 264.705 7.62683

epoch_1500
train 266.437 9.56278

epoch_1600
train 266.322 13.4568

epoch_1700
train 257.557 9.49397

epoch_1800
train 264.206 9.84522

epoch_1900
train 262.44 8.03165

epoch_2000
train 326.061 11.474



In [11]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()